<a href="https://colab.research.google.com/github/langroid/langroid/blob/main/examples/Langroid_QuickStart_OpenAI_Assistants_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent programming with Langroid, using the new OpenAI Assistant API

OpenAI's [Assistants API](https://platform.openai.com/docs/assistants/overview) provides several conveniences to help build LLM applications, such as:
- managing conversation state (threads)
- persistent threads and assistants
- tools (function-calling, retrieval, code-interpreter)


There is a new programming paradigm emerging, where these assistants are primitives, and a key chalenge is:

> how can you have these assistants collaborate to solve a task?

[Langroid](https://github.com/langroid/langroid)'s new `OpenAIAssistant` class offers this ability. Langroid was designed from the start to support a multi-agent LLM programming paradigm, where agents can collaborate on a task via conversation.
The new `OpenAIAssistant` agent gives you:

- 1️⃣ a dead-simple interface to the Assistants API,
- 2️⃣ a seamless way to have assistants collaborate with each other or with users.

The Assistant API fits naturally into Langroid's notion of a `ChatAgent`,
and the `OpenAIAssistant` class derives from `ChatAgent`.
`OpenAIAssistant` can be used as a drop-in replacement for `ChatAgent` in any
Langroid application, and leverage the **multi-agent** task orchestration built
into Langroid.

This notebook takes you on a guided tour of using Langroid's `OpenAIAssistant` from the simplest possible LLM-interaction example, to a two-agent system that extracts structured information from a lease document.

![langroid-oai](https://github.com/langroid/langroid-examples/blob/main/examples/docqa/langroid-oai.png?raw=true)



## Install, setup, import

In [ ]:
# Silently install, suppress all output (~2-4 mins)
!pip install -q --upgrade langroid &> /dev/null
!pip show langroid

In [ ]:
# various unfortunate things that need to be done to
# control notebook behavior.

# (a) output width

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# (b) logging related
import logging
logging.basicConfig(level=logging.ERROR)
import warnings
warnings.filterwarnings('ignore')
import logging
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.ERROR)



#### OpenAI API Key (Needs GPT4-TURBO)

In [ ]:
# OpenAI API Key: Enter your key in the dialog box that will show up below
# NOTE: colab often struggles with showing this input box,
# if so, simply insert your API key in this cell, though it's not ideal.
import os

from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('Enter your GPT4-Turbo-capable OPENAI_API_KEY key:', stream=None)




In [ ]:
from pydantic import BaseModel
import json
import os

from langroid.agent.openai_assistant import (
    OpenAIAssistantConfig,
    OpenAIAssistant,
    AssistantTool,
)

from langroid.agent.chat_agent import ChatAgent, ChatAgentConfig
from langroid.agent.task import Task
from langroid.agent.tool_message import ToolMessage
from langroid.language_models.openai_gpt import OpenAIGPTConfig, OpenAIChatModel
from langroid.utils.logging import setup_colored_logging
from langroid.utils.constants import NO_ANSWER
from langroid.utils.configuration import settings
settings.notebook = True

## Example 1: Basic Chat Example with Assistant API
Langroid's `OpenAIAssistant` class helps you easily use the OpenAI Assistant API to get a response from the LLM and ask follow-up questions (note that conversation state is maintained by the Assistant API via threads).


In [ ]:
cfg = OpenAIAssistantConfig(
    llm = OpenAIGPTConfig(chat_model=OpenAIChatModel.GPT4_TURBO)
)
agent = OpenAIAssistant(cfg)

response = agent.llm_response("What is the square of 3?")

In [ ]:
response = agent.llm_response("What about 5?") # maintains conv state

## Example 2: Wrap Agent in a Task, run it

An `OpenAIAssistant` agent has various capabilities (LLM responses, agent methods/tools, etc) but there is no mechanism to iterate over these capabilities or with a human or with other agents.
This is where the `Task` comes in: Wrapping this agent in a `Task` allows you to run interactive loops with a user or other agents (you will see more examples below).

In [ ]:
task = Task(
    agent,
    system_message="""User will give you a word,
      return its antonym if possible, else say DO-NOT-KNOW.
      Be concise!",
      """,
    single_round=True
)
result = task.run("ignorant")


## Example 3: OpenAIAssistant Agent + Task with Code Interpreter
Here we attach the "code_interpreter" tool (from the OpenAI Assistant API) to the agent defined above, and run it in a task.

In [ ]:
agent.add_assistant_tools([AssistantTool(type="code_interpreter")])
task = Task(agent, interactive=False, single_round=True)
result = task.run("What is the 10th Fibonacci number, if you start with 1,2?")

## Example 4: OpenAIAssistant with Retrieval
Attach a file (a lease document) and the "retrieval" tool, and ask questions about the document.

In [ ]:
# get the lease document

import requests
file_url = "https://raw.githubusercontent.com/langroid/langroid-examples/main/examples/docqa/lease.txt"
response = requests.get(file_url)
with open('lease.txt', 'wb') as file:
    file.write(response.content)

# verify
#with open('lease.txt', 'r') as file:
#   print(file.read())

# now create agent, add retrieval tool and file
agent = OpenAIAssistant(cfg)
agent.add_assistant_tools([AssistantTool(type="retrieval")])
agent.add_assistant_files(["lease.txt"])
response = agent.llm_response("What is the start date of the lease?")


## Example 5: OpenAIAsssistant + Task: Custom Function-calling
You can define your own custom function (or `ToolMessage` in Langroid terminology), enable the agent to use it, and have a special method to handle the message when the LLM emits such a message.

In [ ]:
# Define your own function for the LLM to call;
# this function will be executed by the Langroid agent as part of the task loop

class SquareTool(ToolMessage):
    request = "square"
    purpose = "To find the square of a number <num>"
    num: int

    def handle(self) -> str:
        return str(self.num ** 2)

# create agent, add tool to agent
cfg = OpenAIAssistantConfig(
    llm=OpenAIGPTConfig(chat_model=OpenAIChatModel.GPT4_TURBO),
    name="NumberExpert",
)
agent = OpenAIAssistant(cfg)
agent.enable_message(SquareTool)
task = Task(
    agent,
    system_message="""
    User will ask you to square a number.
    You do NOT know how, so you will use the
    `square` function to find the answer.
    When you get the answer say DONE and show it.
    """,
    interactive=False,
)
response = task.run("What is the square of 5?")


## Example 6: 2-Agent system to extract structured info from a Lease Document
Now we are ready to put together the various notions above, to build a two-agent system where:
- Lease Extractor Agent is required to collect structured information about a lease document, but does not have access to it, so it generates questions to:
- Retriever Agent which answers questions it receives, using the "retrieval" tool, based on the attached lease document


#### Define the desired structure with Pydantic classes

In [ ]:

class LeasePeriod(BaseModel):
    start_date: str
    end_date: str


class LeaseFinancials(BaseModel):
    monthly_rent: str
    deposit: str


class Lease(BaseModel):
    """
    Various lease terms.
    Nested fields to make this more interesting/realistic
    """

    period: LeasePeriod
    financials: LeaseFinancials
    address: str



#### Define the ToolMessage (Langroid's version of function call)

In [ ]:

class LeaseMessage(ToolMessage):
    """Tool/function to use to present details about a commercial lease"""

    request: str = "lease_info"
    purpose: str = "Collect information about a Commercial Lease."
    terms: Lease

    def handle(self):
        """Handle this tool-message when the LLM emits it.
        Under the hood, this method is transplated into the OpenAIAssistant class
        as a method with name `lease_info`.
        """
        print(f"DONE! Successfully extracted Lease Info:" f"{self.terms}")
        return json.dumps(self.terms.dict())

#### Define RetrieverAgent and Task
This agent uses the OpenAI retrieval tool to answer questions based on the attached lease file

In [ ]:
  retriever_cfg = OpenAIAssistantConfig(
        name="LeaseRetriever",
        llm=OpenAIGPTConfig(chat_model=OpenAIChatModel.GPT4_TURBO),
        system_message="Answer questions based on the documents provided.",
    )

  retriever_agent = OpenAIAssistant(retriever_cfg)
  retriever_agent.add_assistant_tools([AssistantTool(type="retrieval")])
  retriever_agent.add_assistant_files(["lease.txt"])

  retriever_task = Task(
      retriever_agent,
      llm_delegate=False,
      single_round=True,
  )

#### Define the ExtractorAgent and Task
This agent is told to collect information about the lease in the desired structure, and it generates questions to be answered by the Retriever Agent defined above.

In [ ]:
    extractor_cfg = OpenAIAssistantConfig(
        name="LeaseExtractor",
        llm=OpenAIGPTConfig(chat_model=OpenAIChatModel.GPT4_TURBO),
        system_message=f"""
        You have to collect information about a Commercial Lease from a
        lease contract which you don't have access to. You need to ask
        questions to get this information. Ask only one or a couple questions
        at a time!
        Once you have all the REQUIRED fields,
        say DONE and present it to me using the `lease_info`
        function/tool (fill in {NO_ANSWER} for slots that you are unable to fill).
        """,
    )
    extractor_agent = OpenAIAssistant(extractor_cfg)
    extractor_agent.enable_message(LeaseMessage, include_defaults=False)

    extractor_task = Task(
        extractor_agent,
        llm_delegate=True,
        single_round=False,
        interactive=False,
    )





#### Add the Retriever as a subtask of Extractor, Run Extractor

In [ ]:
extractor_task.add_sub_task(retriever_task)
extractor_task.run()